In [27]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader 
import torch.optim as optim 
from torchinfo import summary


## How to increase training accuracy

### 1. Network Architectues

#### VGG16

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
block1 = nn.Sequential(
    nn.Conv2d(3,64, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.Conv2d(64,64, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2)
)

block2 = nn.Sequential(
    nn.Conv2d(64,128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.Conv2d(128,128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2)
)

block3 = nn.Sequential(
    nn.Conv2d(128,256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.Conv2d(256,256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.Conv2d(256,256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2)
)

block4 = nn.Sequential(
    nn.Conv2d(256,512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.Conv2d(512,512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.Conv2d(512,512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2)
)

block5 = nn.Sequential(
    nn.Conv2d(512,512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.Conv2d(512,512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.Conv2d(512,512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2)
)

In [10]:
classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512*7*7, 4096), nn.ReLU(inplace=True),
    nn.Linear(4096, 4096), nn.ReLU(inplace=True),
    nn.Linear(4096, 1000),
)

In [11]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.block1 = block1
        self.block2 = block2
        self.block3 = block3
        self.block4 = block4
        self.block5 = block5
        self.classifier = classifier
        
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.classifier(x)
        return x
        

In [14]:
model = VGG16()
model.to(device)

VGG16(
  (block1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool

#### VGG with data

In [90]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(3,64,3, stride=1, padding = 'same')
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(64,64,3, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(64,128,3, stride=1, padding='same'),
            nn.ReLU()
        )
        
        self.conv_layer4 = nn.Sequential(
            nn.Conv2d(128,128, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )   
        
        self.conv_layer5 = nn.Sequential(
            nn.Conv2d(128,256, 3, stride=1, padding='same'),
            nn.ReLU(),
        ) 

        self.conv_layer6 = nn.Sequential(
            nn.Conv2d(256,256, 3, stride=1, padding='same'),
            nn.ReLU(),
        )   
        
        self.conv_layer7 = nn.Sequential(
            nn.Conv2d(256,256, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )   
        
        self.conv_layer8 = nn.Sequential(
            nn.Conv2d(256,512, 3, stride=1, padding='same'),
            nn.ReLU(),
        ) 

        self.conv_layer9 = nn.Sequential(
            nn.Conv2d(512,512, 3, stride=1, padding='same'),
            nn.ReLU(),
        )   
        
        self.conv_layer10 = nn.Sequential(
            nn.Conv2d(512,512, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )   
        
        self.conv_layer11 = nn.Sequential(
            nn.Conv2d(512,512, 3, stride=1, padding='same'),
            nn.ReLU(),
        ) 

        self.conv_layer12 = nn.Sequential(
            nn.Conv2d(512,512, 3, stride=1, padding='same'),
            nn.ReLU(),
        )   
        
        self.conv_layer13 = nn.Sequential(
            nn.Conv2d(512,512, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )   
                 
    def forward(self, x):
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        
        x = self.conv_layer3(x)
        x = self.conv_layer4(x)
        
        x = self.conv_layer5(x)
        x = self.conv_layer6(x)
        x = self.conv_layer7(x)
        
        x = self.conv_layer8(x)
        x = self.conv_layer9(x)
        x = self.conv_layer10(x)
        
        x = self.conv_layer11(x)
        x = self.conv_layer12(x)
        x = self.conv_layer13(x)
        return x


In [91]:
from torchinfo import summary

In [98]:
model = CNNModel()
model = model.to(device)
summary(model, (3, 32,32))

Layer (type:depth-idx)                   Output Shape              Param #
CNNModel                                 [512, 1, 1]               --
├─Sequential: 1-1                        [64, 32, 32]              --
│    └─Conv2d: 2-1                       [64, 32, 32]              1,792
├─Sequential: 1-2                        [64, 16, 16]              --
│    └─Conv2d: 2-2                       [64, 32, 32]              36,928
│    └─ReLU: 2-3                         [64, 32, 32]              --
│    └─MaxPool2d: 2-4                    [64, 16, 16]              --
├─Sequential: 1-3                        [128, 16, 16]             --
│    └─Conv2d: 2-5                       [128, 16, 16]             73,856
│    └─ReLU: 2-6                         [128, 16, 16]             --
├─Sequential: 1-4                        [128, 8, 8]               --
│    └─Conv2d: 2-7                       [128, 16, 16]             147,584
│    └─ReLU: 2-8                         [128, 16, 16]             --

#### Implement VGG16 with Fashion-MNIST dataset

In [93]:
class FashionModel(nn.Module):
    def __init__(self):
        super(FashionModel, self).__init__()
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(1,64,3, stride=1, padding='same'), 
            nn.Sigmoid(),
            nn.MaxPool2d(2,2)
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(64,128,3, stride=1, padding='same'), 
            nn.Sigmoid(),
            nn.MaxPool2d(2,2)
        )
        self.conv_layer3 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*7*7, 512),
            nn.Sigmoid(),
            nn.Linear(512,10)
        )
    def forward(self, x):
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        x = self.conv_layer3(x)
        return x 
    

In [85]:
from torchsummary import summary

In [86]:
model = FashionModel()
model = model.to(device)
summary(model, (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]             640
           Sigmoid-2           [-1, 64, 28, 28]               0
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
           Sigmoid-5          [-1, 128, 14, 14]               0
         MaxPool2d-6            [-1, 128, 7, 7]               0
           Flatten-7                 [-1, 6272]               0
            Linear-8                  [-1, 512]       3,211,776
           Sigmoid-9                  [-1, 512]               0
           Linear-10                   [-1, 10]           5,130
Total params: 3,291,402
Trainable params: 3,291,402
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.35
Params size (MB): 12.56
Estima

In [97]:
from torchinfo import summary
model = FashionModel()
model = model.to(device)
summary(model, input_size=(1, 1, 28, 28))


Layer (type:depth-idx)                   Output Shape              Param #
FashionModel                             [1, 10]                   --
├─Sequential: 1-1                        [1, 64, 14, 14]           --
│    └─Conv2d: 2-1                       [1, 64, 28, 28]           640
│    └─Sigmoid: 2-2                      [1, 64, 28, 28]           --
│    └─MaxPool2d: 2-3                    [1, 64, 14, 14]           --
├─Sequential: 1-2                        [1, 128, 7, 7]            --
│    └─Conv2d: 2-4                       [1, 128, 14, 14]          73,856
│    └─Sigmoid: 2-5                      [1, 128, 14, 14]          --
│    └─MaxPool2d: 2-6                    [1, 128, 7, 7]            --
├─Sequential: 1-3                        [1, 10]                   --
│    └─Flatten: 2-7                      [1, 6272]                 --
│    └─Linear: 2-8                       [1, 512]                  3,211,776
│    └─Sigmoid: 2-9                      [1, 512]                  --
│  

### 2. Network Training

### 3. Case Study

### 4. Problem-Solving Approach